In [1]:
import pyodbc
import pandas as pd
import numpy as np
import time
import logging

from dupdetectorML import DatasetBuilder, Predictor, RestoreModel

Configure parameters 

In [2]:
DATASOURCE_COLUMNS = ['FindCode', 'Name', 'Phone', 'Fax', 'OtherPhone', \
                                  'Email', 'WebsiteURL', 'MailingAddressFreeform', \
                                  'MailingAddressCity', 'MailingAddressPostalCode', 'MailingAddressState']
DATASOURCE_INDEX = 'FindCode'
ONE_HOT_ENCONDING_COLUMNS = {'MailingAddressState': ['nunavut ',
                                                        'saskatchewan',
                                                        'ontario  ',
                                                        'alberta',
                                                        'british columbia ',
                                                        'prince edward island ',
                                                        'yukon territory',
                                                        'newfoundland',
                                                        'northwest territories',
                                                        'new brunswick',
                                                        'manitoba',
                                                        'nova scotia ',
                                                        'quebec']}
TEXT_METRICS = ['ratio','partial_ratio','token_sort_ratio','token_set_ratio','distance',
                'l_ratio','jaro','jaro_winkler','setratio','seqratio','longestnumericseq']
PASS_THROUGH_COMULNS = ['FindCode','MailingAddressState']
HIGH_IMPORTANCE_COLUMNS = ['Name']

alteration_rules = [
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['alpha', 2, 'replace'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [2],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    }
]

COLUMN_ALTERATIION_RULES = []
for i in range(0, len(alteration_rules)):
    COLUMN_ALTERATIION_RULES.append(
        {
            1: alteration_rules[i],
            2: alteration_rules[i],                    
            3: alteration_rules[i],
            4: alteration_rules[i],
            5: alteration_rules[i],
            6: alteration_rules[i],
            7: alteration_rules[i],
            8: alteration_rules[i],
            9: alteration_rules[i]
        })

Fetch data from source. Meta should match configuration above

In [3]:
df_source = pd.read_csv('prediction_input.csv', header=0, names=DATASOURCE_COLUMNS)
df_source.FindCode = df_source.FindCode.astype(str).str.lower()
df_source.Name = df_source.Name.astype(str).str.lower()
df_source.Phone = df_source.Phone.astype(str).str.lower()
df_source.Fax = df_source.Fax.astype(str).str.lower()
df_source.OtherPhone = df_source.OtherPhone.astype(str).str.lower()
df_source.Email = df_source.Email.astype(str).str.lower()
df_source.WebsiteURL = df_source.WebsiteURL.astype(str).str.lower()
df_source.MailingAddressFreeform = df_source.MailingAddressFreeform.astype(str).str.lower()
df_source.MailingAddressCity = df_source.MailingAddressCity.astype(str).str.lower()
df_source.MailingAddressPostalCode = df_source.MailingAddressPostalCode.astype(str).str.lower()
df_source.MailingAddressState = df_source.MailingAddressState.astype(str).str.lower()
df_source.replace(to_replace='none', value='', inplace=True)

df_what_to_search = df_source[0:10]
df_where_to_search = df_source[10:]



Create predictiondataset

In [4]:
builder = DatasetBuilder(DATASOURCE_COLUMNS, DATASOURCE_INDEX, 
                         ONE_HOT_ENCONDING_COLUMNS, TEXT_METRICS, 
                         PASS_THROUGH_COMULNS, COLUMN_ALTERATIION_RULES,
                         HIGH_IMPORTANCE_COLUMNS, 8, logging.DEBUG)
predicting_df = builder.generatePredictionDataset(df_what_to_search, df_where_to_search)

C:\Users\masya\MyProjects\DeduplicatorML\datasetbuilder\datasetbuilder.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df.apply(lambda row: duplicatePattern(row[column], word), axis=1)
C:\Users\masya\MyProjects\DeduplicatorML\datasetbuilder\datasetbuilder.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df.apply(lambda row: duplicateNumericSequence(row[column]), axis=1)
2020-04-18 14:11:21,793 - root - INFO - Column: WebsiteURL_x, ratio metric took: 2.2492456436157227 seco

2020-04-18 14:13:35,496 - root - INFO - Column: MailingAddressFreeform_x, setratio metric took: 2.2142951488494873 seconds
2020-04-18 14:13:37,641 - root - INFO - Column: MailingAddressFreeform_x, seqratio metric took: 2.1440653800964355 seconds
2020-04-18 14:13:39,671 - root - INFO - Column: MailingAddressFreeform_x, longestnumericseq metric took: 2.030172109603882 seconds
2020-04-18 14:13:41,951 - root - INFO - Column: Name_x, ratio metric took: 2.267970561981201 seconds
2020-04-18 14:13:44,170 - root - INFO - Column: Name_x, partial_ratio metric took: 2.217057704925537 seconds
2020-04-18 14:13:46,436 - root - INFO - Column: Name_x, token_sort_ratio metric took: 2.2644505500793457 seconds
2020-04-18 14:13:48,505 - root - INFO - Column: Name_x, token_set_ratio metric took: 2.069516181945801 seconds
2020-04-18 14:13:50,806 - root - INFO - Column: Name_x, distance metric took: 2.3000681400299072 seconds
2020-04-18 14:13:53,163 - root - INFO - Column: Name_x, l_ratio metric took: 2.35558

In [5]:
model = RestoreModel("TrainedModel.sav")
predictor = Predictor(model)
predictor.execute(predicting_df)

In [11]:
matches = builder.getCompleteDataset()[
    [e + '_x' for e in DATASOURCE_COLUMNS] + [e + '_y' for e in DATASOURCE_COLUMNS]].iloc[predictor.getPrediction().index]
matches[1:2]

,FindCode_x,Name_x,Phone_x,Fax_x,OtherPhone_x,Email_x,WebsiteURL_x,MailingAddressFreeform_x,MailingAddressCity_x,MailingAddressPostalCode_x,...,Name_y,Phone_y,Fax_y,OtherPhone_y,Email_y,WebsiteURL_y,MailingAddressFreeform_y,MailingAddressCity_y,MailingAddressPostalCode_y,MailingAddressState_y
997,aabhx,bill priddle,5199236943,,,,,rr 1,badjeros,n0c1a0,...,john veeke,5198283313,,,,,rr 1,arkona,n0m1b0,ontario
